## Ramps-to-slopes:  Detector1Pipeline   (MIRI CALDETECTOR1)

The `Detector1Pipeline` takes DMS Level 1B data and applies basic detector-level corrections to all exposure types. It is applied to one exposure at a time. It is sometimes referred to as “ramps-to-slopes” processing, because the input raw data are in the form of one or more ramps (integrations) containing accumulating counts from the non-destructive detector readouts and the output is a corrected countrate (slope) image.

Official documentation for `Detector1Pipeline` can be found here:

<https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html>

The `Detector1Pipelin`e comprises a linear series of steps to first calibrate the pixel ramps, detect cosmic ray jumps, and fit a linear function to the calibrate ramps to produce a slope or DMS Level 2A image. The steps applied to MIRI data in order are:

|Step|Description|
|:---|:---|
|`dq_init`|populates the DQ mask for the input dataset|
|`saturation`|flags saturated pixel values|
|`firstframe`|flags the first group in every integration as bad|
|`lastframe`|flags the last group in every integration as bad|
|`linearity`|corrects for detector non-linearity|
|`rscd`|correct for the reset switch charge decay|
|`dark_current`|subtracts dark current data|
|`refpix`|corrects for pixel drifts using the reference pixels|
|`jump`|detects jumps in the ramps by looking for outliers in each integration|
|`ramp_fit`|determines count rate for each pixel by performing a linear fit to ramps|

For more information and examples of each of the steps click on the links in the side bar.

### Input data

An example of running the file through the Detector1Pipeline is now shown. The file is a simple simulated observation of a point source with the F1130W filter produced with [MIRISim v2.1](http://miri.ster.kuleuven.be/bin/view/Public/MIRISimPublicRelease2dot1).

### Python

Start by importing what will be used, disable warnings and set the `CRDS_CONTEXT` to use

In [1]:
# imports
import os, glob, shutil
import numpy as np
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from subprocess import call
from jwst import datamodels

import warnings
warnings.filterwarnings('ignore')

# set the CRDS_CONTEXT
os.environ["CRDS_CONTEXT"] = "jwst_0535.pmap"

Import Detector1Pipeline and print the docstring to show some information

In [2]:
from jwst.pipeline import Detector1Pipeline
print(Detector1Pipeline.__doc__)


    Detector1Pipeline: Apply all calibration steps to raw JWST
    ramps to produce a 2-D slope product. Included steps are:
    group_scale, dq_init, saturation, ipc, superbias, refpix, rscd,
    lastframe, linearity, dark_current, persistence, jump detection,
    ramp_fit, and gain_scale.
    


Set the name of the input fil, where the output should be saved and run the pipeline. The output level 2A files will be saved in my_output_dir as `_rate.fits`. Note that we must explicitly skip the IPC step when running from Python. This is achieved by passing the `skip` parameter to the step in a Python `dict`.

*Parameters used:*

`output_use_model` : boolean, optional, default=False  
&nbsp;&nbsp;&nbsp;&nbsp; propagate the input filename to the output
    
`save_results`: boolean, optional, default=False  
&nbsp;&nbsp;&nbsp;&nbsp; save the results to file
    
`output_dir` : boolean, optional, default is the working directory   
&nbsp;&nbsp;&nbsp;&nbsp; the location to save the output
    
`steps` : dict, optional  
&nbsp;&nbsp;&nbsp;&nbsp; dict of user-defined parameters for individual steps  

Note that the Detector1Pipeline will return the rate datamodel so we set this to the `dm` variable.


In [ ]:
# user specified
my_input_file = 'det_image_seq1_MIRIMAGE_F1130Wexp1.fits'
my_output_dir = 'demo_output'

# the output directory should be created if it doesn't exist
if not os.path.exists(my_output_dir): 
    os.mkdir(my_output_dir)

# run the pipeline
dm = Detector1Pipeline.call(my_input_file, output_use_model=True, save_results=True, 
                            output_dir=my_output_dir, steps={'ipc': {'skip': True}})

2019-07-01 15:50:37,345 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2019-07-01 15:50:37,347 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2019-07-01 15:50:37,349 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2019-07-01 15:50:37,351 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2019-07-01 15:50:37,352 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2019-07-01 15:50:37,354 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2019-07-01 15:50:37,356 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2019-07-01 15:50:37,358 - stpipe.Detector1Pipeline.rscd - INFO - RSCD_Step instance created.
2019-07-01 15:50:37,361 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2019-07-01 15:50:37,363 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2019-07-01 

2019-07-01 15:51:02,853 - CRDS - INFO -  Fetching  /Users/patrickkavanagh/crds_mirror/references/jwst/miri/jwst_miri_readnoise_0057.fits    4.2 M bytes  (1 / 1 files) (0 / 4.2 M bytes)
2019-07-01 15:51:09,548 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/Users/patrickkavanagh/crds_mirror/references/jwst/miri/jwst_miri_dark_0058.fits'.
2019-07-01 15:51:09,549 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/Users/patrickkavanagh/crds_mirror/references/jwst/miri/jwst_miri_gain_0008.fits'.
2019-07-01 15:51:09,550 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/patrickkavanagh/crds_mirror/references/jwst/miri/jwst_miri_linearity_0024.fits'.
2019-07-01 15:51:09,551 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/Users/patrickkavanagh/crds_mirror/references/jwst/miri/jwst_miri_mask_0023.fits'.
2019-07-01 15:51:09,552 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT refer

2019-07-01 15:52:14,006 - stpipe.Detector1Pipeline.jump - INFO - Step jump done
2019-07-01 15:52:14,067 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit running with args (<RampModel(2, 50, 1024, 1032) from det_image_seq1_MIRIMAGE_F1130Wexp1.fits>,).
2019-07-01 15:52:14,085 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /Users/patrickkavanagh/crds_mirror/references/jwst/miri/jwst_miri_readnoise_0057.fits
2019-07-01 15:52:14,200 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /Users/patrickkavanagh/crds_mirror/references/jwst/miri/jwst_miri_gain_0008.fits
2019-07-01 15:52:14,216 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2019-07-01 15:52:14,217 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2019-07-01 15:52:14,241 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2019-07-01 15:52:14,245 - stpipe.Detector1Pipeline.ramp_fit -

We can plot the before (last frame of first integration) and after images

In [ ]:
# open the input image as a jwst data model
with datamodels.open(my_input_file) as in_dm:

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    axs[0].imshow(in_dm.data[0,-1,:,:], cmap='jet', interpolation='nearest', origin='lower', norm=LogNorm(vmin=1e4,vmax=2e4))
    axs[0].annotate('Input ramp image (last frame)', xy=(0.0, 1.02), xycoords='axes fraction', fontsize=12, fontweight='bold', color='k')
    axs[0].set_facecolor('black')
    axs[1].imshow(out_dm.data, cmap='jet', interpolation='nearest', origin='lower', norm=LogNorm(vmin=2, vmax=100))
    axs[1].annotate('Calibrated slope image', xy=(0.0, 1.02), xycoords='axes fraction', fontsize=12, fontweight='bold', color='k')
    axs[1].set_facecolor('black')
    plt.tight_layout()
    plt.show()


### Command line

To achieve the same result from the command line we must first collect the pipeline configuration files and then run the `Detector1Pipeline` using the `strun` command:

```bash
mkdir demo_output

collect_pipeline_cfgs cfgs/

strun cfgs/calwebb_detector1.cfg det_image_seq1_MIRIMAGE_F1130Wexp1.fits --output_dir demo_output
```

This will produce the same output file in the user-defined `--output_dir`


## Further examples

Other notebooks with more complex examples can be found here:

